In [1]:
from __future__ import print_function
from six.moves import cPickle as pickle
import tensorflow as tf

In [2]:
f=open('Data.pickle','rb')
Data=open('Data.pickle','rb')
train_dataset=pickle.load(Data)['train_dataset']
Data.seek(0)
train_labels=pickle.load(Data)['train_labels']
Data.seek(0)
test_dataset=pickle.load(Data)['test_dataset']
Data.seek(0)
test_labels=pickle.load(Data)['test_labels']
Data.close()

In [3]:
print(train_dataset.shape)

(50000, 28, 28)


In [4]:
train_dataset=train_dataset.reshape(train_dataset.shape[0],-1).T
train_labels=train_labels.reshape(train_labels.shape[0],-1).T
test_dataset=train_dataset.reshape(test_dataset.shape[0],-1).T
test_labels=train_labels.reshape(test_labels.shape[0],-1).T

In [5]:
def One_hot_encoder(labels,C):
    C=tf.constant(C,dtype=tf.int32,name='C')
    encoded=tf.one_hot(labels,C)
    with tf.Session() as sess:
        return sess.run(encoded)

In [6]:
Train_labels=One_hot_encoder(train_labels,10)
#rint(Train_labels.shape)
Train_labels=Train_labels[0]
print(Train_labels.shape)
#rint(train_labels[0])
#rint(Train_labels[0][0])
print(train_dataset.shape)

(50000, 10)
(784, 50000)


In [7]:
total_layers=12
Layer_Units={1:28,2:28,3:28,4:28,5:28,6:28,7:28,8:28,9:28,10:28,11:28}
Layer_Units[total_layers]=10
cache={}
for i in range(1,total_layers+1):
    cache['Z'+str(i)]=0
    cache['A'+str(i)]=0
parameters={}
count=1
for i in Layer_Units:
    num=i
    weight="W"+str(num)
    bias='b'+str(num)
    if i==1:
        parameters[weight]=tf.get_variable(weight,[Layer_Units[i],784],initializer=tf.contrib.layers.xavier_initializer(seed=1))
        parameters[bias] = tf.get_variable(bias, [Layer_Units[i], 1], initializer = tf.zeros_initializer())
    else:
        parameters[weight]=tf.get_variable(weight,[Layer_Units[i],Layer_Units[i-1]],initializer=tf.contrib.layers.xavier_initializer(seed=1)) 
        parameters[bias] = tf.get_variable(bias, [Layer_Units[i], 1], initializer = tf.zeros_initializer())

In [8]:
for i in Layer_Units:
        num=i
        if i==1:
            cache['Z'+str(i)]=tf.add(tf.matmul(parameters['W'+str(num)],train_dataset),parameters['b'+str(num)])
            cache['A'+str(i)]=tf.nn.relu(cache['Z'+str(i)])   
        elif i==total_layers:
            cache['Z'+str(i)]=tf.add(tf.matmul(parameters['W'+str(num)],cache['A'+str(i-1)]),parameters['b'+str(num)])
        else:
            cache['Z'+str(i)]=tf.add(tf.matmul(parameters['W'+str(num)],cache['A'+str(i-1)]),parameters['b'+str(num)])
            cache['A'+str(i)]=tf.nn.relu(cache['Z'+str(i)])

In [9]:
logits = tf.transpose(cache['Z'+str(total_layers)])
labels = Train_labels
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels =labels))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
count=1
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init)
    print('Number','Loss','Accuracy')
    for i in range(1000):
        session.run(train_op)
        loss, acc = session.run([cost, accuracy])
        print(count,':',loss,':',acc)
        count=count+1   

Number Loss Accuracy
1 : 2.30566 : 0.09996
2 : 2.30153 : 0.11528
3 : 2.30043 : 0.0942
4 : 2.30179 : 0.10238
5 : 2.29809 : 0.10218
6 : 2.28464 : 0.12334
7 : 2.2768 : 0.13586
8 : 2.29942 : 0.1035
9 : 2.29171 : 0.10344
10 : 2.24108 : 0.19106
11 : 2.24801 : 0.16108
12 : 2.19122 : 0.17018
13 : 2.1079 : 0.18004
14 : 2.09394 : 0.2053
15 : 2.07046 : 0.18842
16 : 2.06676 : 0.21648
17 : 1.99264 : 0.1849
18 : 2.0293 : 0.19116
19 : 1.94994 : 0.2244
20 : 1.97517 : 0.22882
21 : 1.9332 : 0.22408
22 : 1.92227 : 0.19042
23 : 1.87916 : 0.28076
24 : 1.86869 : 0.27962
25 : 1.83376 : 0.30816
26 : 1.82076 : 0.32542
27 : 1.7755 : 0.35074
28 : 1.77049 : 0.32936
29 : 1.73512 : 0.31562
30 : 1.74651 : 0.30746
31 : 1.76313 : 0.32036
32 : 1.77497 : 0.33856
33 : 1.67325 : 0.36424
34 : 1.69014 : 0.35516
35 : 1.62934 : 0.40162
36 : 1.63287 : 0.39716
37 : 1.57953 : 0.39892
38 : 1.55702 : 0.41434
39 : 1.56123 : 0.39864
40 : 1.61222 : 0.40332
41 : 1.4866 : 0.45738
42 : 1.52605 : 0.44208
43 : 1.49516 : 0.4422
44 : 1.4770

341 : 0.450213 : 0.8433
342 : 0.449258 : 0.84382
343 : 0.453524 : 0.84242
344 : 0.457712 : 0.8406
345 : 0.4504 : 0.84312
346 : 0.450914 : 0.84296
347 : 0.454404 : 0.84156
348 : 0.450702 : 0.84292
349 : 0.44809 : 0.84342
350 : 0.452857 : 0.8418
351 : 0.449465 : 0.84338
352 : 0.445391 : 0.8447
353 : 0.443872 : 0.84476
354 : 0.44761 : 0.84408
355 : 0.453867 : 0.84158
356 : 0.449125 : 0.84384
357 : 0.451107 : 0.84184
358 : 0.459487 : 0.83886
359 : 0.47381 : 0.83482
360 : 0.471644 : 0.83454
361 : 0.470699 : 0.83548
362 : 0.450033 : 0.84254
363 : 0.462872 : 0.83782
364 : 0.476713 : 0.83296
365 : 0.45236 : 0.84188
366 : 0.451434 : 0.84274
367 : 0.455046 : 0.8407
368 : 0.445619 : 0.84446
369 : 0.449402 : 0.84278
370 : 0.445938 : 0.84376
371 : 0.444404 : 0.84566
372 : 0.450811 : 0.8423
373 : 0.447473 : 0.84332
374 : 0.444655 : 0.84404
375 : 0.458657 : 0.84024
376 : 0.463191 : 0.83772
377 : 0.456074 : 0.84058
378 : 0.456024 : 0.84092
379 : 0.446594 : 0.84402
380 : 0.438565 : 0.84676
381 : 0.4451

674 : 0.39561 : 0.85986
675 : 0.395521 : 0.85986
676 : 0.395302 : 0.86028
677 : 0.395093 : 0.8601
678 : 0.395823 : 0.8599
679 : 0.39704 : 0.85842
680 : 0.400913 : 0.8579
681 : 0.406604 : 0.85486
682 : 0.406522 : 0.85528
683 : 0.403612 : 0.8558
684 : 0.39578 : 0.85956
685 : 0.3944 : 0.86034
686 : 0.400348 : 0.85706
687 : 0.398622 : 0.8585
688 : 0.394366 : 0.8596
689 : 0.392451 : 0.86078
690 : 0.394753 : 0.86004
691 : 0.396212 : 0.85842
692 : 0.39314 : 0.86084
693 : 0.390797 : 0.86116
694 : 0.392399 : 0.86022
695 : 0.394018 : 0.86022
696 : 0.394889 : 0.85894
697 : 0.393175 : 0.86038
698 : 0.392656 : 0.86022
699 : 0.392317 : 0.8606
700 : 0.394539 : 0.85948
701 : 0.400372 : 0.85672
702 : 0.411647 : 0.85334
703 : 0.434606 : 0.84526
704 : 0.445295 : 0.84274
705 : 0.438904 : 0.8434
706 : 0.441139 : 0.84332
707 : 0.478214 : 0.83238
708 : 0.423435 : 0.8495
709 : 0.416286 : 0.8511
710 : 0.431101 : 0.84664
711 : 0.406538 : 0.85556
712 : 0.423234 : 0.84852
713 : 0.412752 : 0.85258
714 : 0.414127 :